#Lecture 2 : Code algorithms to classify the behaviors from acceleration data*
<br>

*The data were collected as part of the research projects of Damien Chevallier, Centre National de la Recherche Scientifique (CNRS France)

<br>

lorene@aims.ac.za

African Institute for Mathematical Sciences

Machine Learning for Ecology Research Group



Get the green turtle dataset. To do this, you need to make a shortcut to a shared Google Drive directory in your Google Drive.

1.   Go to the shared Google Drive: [https://drive.google.com/drive/folders/1ltL_AUrVjsCI8bpNs8YwRkxgIe12TRBH?usp=sharing](https://drive.google.com/drive/folders/1ltL_AUrVjsCI8bpNs8YwRkxgIe12TRBH?usp=sharing)
2.   Select "Add shortcut to Drive" and select "My Drive".


## Download the preprocessed data files

In [1]:
# connect to google drive via your google account
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
%cd /content/drive/MyDrive/Shortcut to courses materials/Deep Learning for ecology/

/content/drive/MyDrive/Shortcut to courses materials/Deep Learning for ecology


In [16]:
%ls

2D_CNN_model.h5                              data_y_new.npy     test_5.svl
AnnotationReader.py                          data_y.npy         train_X_final_nn.npy
Code.zip                                     model_arch.png     train_X_final.npy
Data/                                        Preprocessing.py   train_x.npy
Data_Acceleration_GreenTurtle@               __pycache__/       train_Y_final_nn.npy
Data_Acceleration_GreenTurtle_preprocessed@  saved_model.h5     train_Y_final.npy
data_x_final.npy                             spec_data.npy      train_y.npy
data_x_new.npy                               spec_final_nn.npy  Y_cat.npy
data_x.npy                                   spec_final.npy
data_y_final.npy                             test_4.svl


## Librairies

In [3]:
import json
import glob
import os
import datetime
import shutil
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
import keras
import scipy
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

## Create a memo to save all the parameters of the different experiments

In [7]:
memo=pd.Series()

<ipython-input-7-88f59b1163a2>:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  memo=pd.Series()


## Function to create a folder to save the results

In [5]:
def create_new_folder(folder_out, model):

        now = datetime.datetime.now()
        today=now.strftime("%Y_%m_%d")
        dir_out=folder_out+"/model="+today+"_"+model


        if os.path.exists(dir_out):
            shutil.rmtree(dir_out)
            print("we clear the directory:",dir_out)
        else:
            print("we create the directory:",dir_out)

        """création des dossiers """
        os.makedirs(dir_out)
        return dir_out

## Load the data

#### load first the dictionary containing the information about the data

In [8]:
FOLDER_NAME=memo['Folder_data']="Data_Acceleration_GreenTurtle_preprocessed"

In [10]:
def load_dico(path,key_int=False,print_me=False):
    with open(path) as f:
        dico_str = json.loads(json.load(f))

    if key_int:
        conv_key=lambda k:int(k)
    else:
        conv_key=lambda k:k

    dico={conv_key(k):v for k,v in dico_str.items()}

    if print_me:
        print(dico)

    return dico

In [17]:
dico_info=load_dico(FOLDER_NAME+"/dico_info.json",print_me=True);
behName_to_behInd=load_dico(FOLDER_NAME+ "/behName_to_behInd_cat.json",print_me=True);
behInd_to_behName=load_dico(FOLDER_NAME+"/behInd_to_behName_cat.json",key_int=True,print_me=True)

{'RESAMPLING_STEP_S': 0.05, 'col_names': ['Accelerometer X [m/s²]', 'Accelerometer Y [m/s²]', 'Accelerometer Z [m/s²]', 'Gyroscope X [mrad/s]', 'Gyroscope Y [mrad/s]', 'Gyroscope Z [mrad/s]', 'Depth [m]', 'beh', 'beh_merge', 'StX', 'StY', 'StZ', 'DX', 'DY', 'DZ', 'VeDBA', 'Pressure_corr', 'Pressure_diff']}
{'Other': 0, 'Breathing': 1, 'Feeding': 2, 'Gliding': 3, 'Resting': 4, 'Scratching': 5, 'Swimming': 6}
{0: 'Other', 1: 'Breathing', 2: 'Feeding', 3: 'Gliding', 4: 'Resting', 5: 'Scratching', 6: 'Swimming'}


In [ ]:
CLASSES=['Other', 'Breathing', 'Feeding', 'Gliding', 'Resting', 'Scratching', 'Swimming']

#### load the matrix of each individual (n=13)

There are 16 different files from 13 different individuals. Each matrix contains the annotated acc/gyro/pression sequences plus various calculated variables.

In [18]:
#get the name of the files
GT_names=[i.split("/")[1][:-4] for i in glob.glob(FOLDER_NAME+'/*.npy')]

In [19]:
GT_names

['CC-07-48_18-02-2019',
 'CC-07-48_08-04-2019_1',
 'CC-07-48_22-05-2018',
 'CC-07-48_26-05-2018',
 'CC-07-107_11-02-2019',
 'CC-07-48_08-10-2018',
 'CC-07-115_14-05-2019',
 'CC-07-48_08-04-2019_3',
 'CC-07-48_04-10-2018',
 'CC-07-115_13-05-2019',
 'CC-07-48_10-10-2018',
 'CC-07-48_08-04-2019_2',
 'CC-07-48_06-10-2018',
 'CC-07-47_14-02-2018',
 'CC-07-59_20-05-2018',
 'CC-07-108_11-02-2019']

In [20]:
#based on the dictionary , we can see the name of the columns of our numpy matrix
for i, name in enumerate(dico_info['col_names']):
  print(i, name)


0 Accelerometer X [m/s²]
1 Accelerometer Y [m/s²]
2 Accelerometer Z [m/s²]
3 Gyroscope X [mrad/s]
4 Gyroscope Y [mrad/s]
5 Gyroscope Z [mrad/s]
6 Depth [m]
7 beh
8 beh_merge
9 StX
10 StY
11 StZ
12 DX
13 DY
14 DZ
15 VeDBA
16 Pressure_corr
17 Pressure_diff


In [21]:
#here we are going to select the columns that we want to use for our analysis
DESCRIPTORS_SELEC=[0,1,2,3,4,5,9,10,11,12,13,14,15,16]
LABEL_COL=8  #here I am indicating the column to use as label

In [22]:
#a cell just to check which columns we are going to use based on DESCRIPTORS_SELEC
DESC_NAMES_SELEC=[]
for i,name in enumerate(dico_info['col_names']):
    if i in DESCRIPTORS_SELEC:
        prefix=""
        DESC_NAMES_SELEC.append(name)
    else:
        prefix="-----suppressed----->"
    print(prefix,i,name)

 0 Accelerometer X [m/s²]
 1 Accelerometer Y [m/s²]
 2 Accelerometer Z [m/s²]
 3 Gyroscope X [mrad/s]
 4 Gyroscope Y [mrad/s]
 5 Gyroscope Z [mrad/s]
-----suppressed-----> 6 Depth [m]
-----suppressed-----> 7 beh
-----suppressed-----> 8 beh_merge
 9 StX
 10 StY
 11 StZ
 12 DX
 13 DY
 14 DZ
 15 VeDBA
 16 Pressure_corr
-----suppressed-----> 17 Pressure_diff


In [23]:
#now we load the matrices, we are going to load them into a dictionary associated with the name of the file/individual
#we do the same for the labels

Matrices={}
Labels={}
FREQ=1/dico_info['RESAMPLING_STEP_S']


for name in GT_names:
        print(name)
        full_name=FOLDER_NAME+"/"+name+".npy"
        if not os.path.exists(full_name):
            assert 1==0,"the file:"+full_name+" doesn't exist"
        else:
            full_mat=np.load(full_name)
            label= full_mat[:,LABEL_COL]

            mat=full_mat[:,DESCRIPTORS_SELEC]
            print("loading the matrice:"+name)
            print("shape:",mat.shape)
            print("Time in seconds:",len(mat)/FREQ)
            print("Time in hours:",round(len(mat)/FREQ/60/60,2))
            print("              ")
            Matrices[name]=mat
            Labels[name]=label


CC-07-48_18-02-2019
loading the matrice:CC-07-48_18-02-2019
shape: (559219, 14)
Time in seconds: 27960.95
Time in hours: 7.77
              
CC-07-48_08-04-2019_1
loading the matrice:CC-07-48_08-04-2019_1
shape: (157315, 14)
Time in seconds: 7865.75
Time in hours: 2.18
              
CC-07-48_22-05-2018
loading the matrice:CC-07-48_22-05-2018
shape: (286161, 14)
Time in seconds: 14308.05
Time in hours: 3.97
              
CC-07-48_26-05-2018
loading the matrice:CC-07-48_26-05-2018
shape: (267301, 14)
Time in seconds: 13365.05
Time in hours: 3.71
              
CC-07-107_11-02-2019
loading the matrice:CC-07-107_11-02-2019
shape: (346142, 14)
Time in seconds: 17307.1
Time in hours: 4.81
              
CC-07-48_08-10-2018
loading the matrice:CC-07-48_08-10-2018
shape: (535201, 14)
Time in seconds: 26760.05
Time in hours: 7.43
              
CC-07-115_14-05-2019
loading the matrice:CC-07-115_14-05-2019
shape: (325009, 14)
Time in seconds: 16250.45
Time in hours: 4.51
              
CC-07-4

In [24]:
Matrices[GT_names[0]].shape

(559219, 14)

In [25]:
print(Labels[GT_names[0]])
print(Labels[GT_names[0]].shape)
print(np.unique(Labels[GT_names[0]], return_counts=True))

[0. 0. 0. ... 6. 6. 6.]
(559219,)
(array([0., 1., 2., 3., 4., 5., 6.]), array([  4687,  32336,    128,  12175, 351587,   3532, 154774]))


## Split our data into Training/Validation dataset

There is two ways to do it :
* do it randomly
* carefully select individuals to ensure that all behaviours are well represented in each data set. This is the difficulty of small data sets in ecology

<br>

Very important : to test the ability of the algorithm to perform on new individuals, the individuals must be distributed separately between the three data sets.



In [ ]:
NAMES_TRAIN=memo["NAMES_TRAIN"]=[
                     'CC-07-48_18-02-2019', #1
                     'CC-07-48_06-10-2018', #2
                     'CC-07-48_04-10-2018', #3
                     'CC-07-115_13-05-2019', #4
                     'CC-07-115_14-05-2019', #4
                     'CC-07-48_08-04-2019_1', #5
                     'CC-07-48_08-04-2019_2', #5
                     'CC-07-48_08-04-2019_3', #5
                     'CC-07-48_22-05-2018', #6
                      'CC-07-108_11-02-2019', #7


                    ]

NAMES_VAL=memo["NAMES_VAL"]=[
    'CC-07-59_20-05-2018', #8
    'CC-07-47_14-02-2018', #9
    'CC-07-48_10-10-2018',  #10

    ]

NAMES_TEST=memo["NAMES_TEST"]=[
    'CC-07-48_26-05-2018', #11
    'CC-07-107_11-02-2019', #12
    'CC-07-48_08-10-2018', #13
    ]


NAMES_ALL=NAMES_TRAIN+NAMES_VAL+NAMES_TEST

## Deep Learning : CNN

### Generate windows

We first need to build the dataset that we are going to use to train the model

### CNN architecture

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend, Input, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, MaxPool1D, Conv1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras as keras



In [ ]:
input_shape=(40,14)
# Starting point
model = keras.Sequential()

# Create one convolutional layer
model.add(keras.layers.Conv1D(filters = 128, kernel_size = 5, input_shape = input_shape, activation = 'relu'))
# Create one max pooling layer
model.add(keras.layers.MaxPool1D(pool_size =4))

# Create another convolutional layer
model.add(keras.layers.Conv1D(filters =128 , kernel_size = 5, activation = 'relu'))
# Create another max pooling layer
model.add(keras.layers.MaxPooling1D(pool_size=4))

model.add(Flatten())

# Here we create 100 fully connected units
model.add(Dense(units = 100, activation='relu'))

# The output
model.add(Dense(7, activation = 'softmax'))

# Here we tell the model how to improve/update the weights
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

### Save the model


### Evaluate the CNN

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize, axe,
                          title,
                          cmap="jet",
                          precision=2,
                         ):


    y_true=y_true.astype(int)
    y_pred=y_pred.astype(int)

    np.set_printoptions(precision=precision)


    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Only use the labels that appear in the data
    print("Confusion matrix, without normalization:")
    print(cm)

    if normalize:
      if axe=="col" :
        cm = cm.astype('float') / cm.sum(axis=0)[np.newaxis, :]
      elif axe=="row":
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
        print(cm)


    fig, ax = plt.subplots(figsize=(12,12))
    ax.grid(False)
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.'+str(precision)+'f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "coral")
    fig.tight_layout()
